# Mail

In [78]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders


def mail(rider_name, location, delivery_number, pick_delivery, nbr_items, img_link):
    mail_content = '''
    <head>
        <p> Rider name: {} </p>
        <p> Location: {} </p>
        <p> Delivery number: {} </p>
        <p> Trip type: {} </p>
        <p> Number of items: {} </p>
    </head>
    <img src={} width="200"  height="200" > </img>
    '''.format(rider_name, location, delivery_number, pick_delivery, nbr_items, img_link)

    # Infos
    sender_address = 'Johninalegwu11@gmail.com'
    sender_pass = 'mvfsnoxuaypekqrv'
    receiver_address = 'ochooigocho8991@gmail.com'

    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = "Delivery Number: {}".format(delivery_number)
    message.attach(MIMEText(mail_content, 'html'))

    #Create SMTP session for sending the mail
    session = smtplib.SMTP_SSL('smtp.gmail.com', 465) #use gmail with port
    # session.starttls() #enable security
    session.ehlo()
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
    print('Mail Sent')

# Libraries

In [79]:
from telegram.ext.updater import Updater
from telegram.update import Update
from telegram.ext.callbackcontext import CallbackContext
from telegram.ext.commandhandler import CommandHandler
from telegram.ext.messagehandler import MessageHandler
from telegram.ext.filters import Filters
from telegram.ext import ConversationHandler
from telegram import ReplyKeyboardMarkup
from telegram import ReplyKeyboardRemove
import logging
import os, sys
from datetime import datetime
import locale
import pandas as pd
import gspread
import gspread_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import telegram
from telegram.ext import Updater, MessageHandler, Filters
from geopy.geocoders import Nominatim

import requests

In [84]:
# Variables
BOT_KEY = '5465907907:AAGFNqtPdomFFo9eev7WsappwOdh7OKqnus'

# Set your path:
path = 'C:\\Users\\JohnInalegwu\\Downloads\\'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive']

# Authenticate using your credentials, saved in JSON in Step 1:
creds = ServiceAccountCredentials.from_json_keyfile_name(path + '/creds.json', scope)

# Open connection with Google Sheets
client = gspread.authorize(creds)
sheet = client.open('sample').sheet1

# Get data as DataFrame
data = gspread_dataframe.get_as_dataframe(sheet, parse_dates=True, usecols=[0,1,2,3,4,5,6,7,8])

#------------------------------------------------------------------------------------------------------
# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)
 
    
# BOT functions
def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('''Hello ! I am your RPL Logistics tracking bot, please find below commands to interract with me. \n /start_tracking: start tracking process''')

# Location Tracking
LOC, DN, P_D, N_I, PHOTO = range(5)


def start_tracking(update: Update, context: CallbackContext):
    user = update.message.from_user
    logger.info("Driver: %s", user.first_name)
    update.message.reply_text(f'Welcome {user.first_name}! Please share your current GPS location.',
        reply_markup = ReplyKeyboardRemove(),
    )
    return LOC

def location(update: Update, context: CallbackContext):
    user = update.message.from_user
    user_location = update.message.location
    logger.info("Driver ID location %s: (%s, %s)", user.first_name, user_location.latitude, user_location.longitude)
    global gps_location
    gps_location = "({}, {})".format(user_location.latitude, user_location.longitude)
    update.message.reply_text(f'{user.first_name}, your current GPS location has been recorded. Please enter delivery number.')
    return DN

def delivery_number(update: Update, context: CallbackContext):
    user = update.message.from_user
    global delivery_numbers
    delivery_numbers = update.message.text
    logger.info("Delivery Number: %s", delivery_numbers)
    update.message.reply_text('Delivery number is now recorded. Please is this a Pick Up or a Delivery?' )
    return P_D

def pick_delivery(update: Update, context: CallbackContext):
    user = update.message.from_user
    global pick_deliveries
    pick_deliveries = update.message.text
    logger.info("Job Type: %s", pick_deliveries)
    update.message.reply_text(f'Job Type has been updated. Please how many items did you pickup/deliver at this location?')
    return N_I

def nbr_item(update: Update, context: CallbackContext):
    user = update.message.from_user
    global nbr_itemss
    nbr_itemss = update.message.text
    logger.info("Number of items: %s", nbr_itemss)
    if int(nbr_itemss) == 1:
        update.message.reply_text(f'You picked {nbr_itemss} item from this location. Please upload a picture of this item.')
    else:
        update.message.reply_text(f'You picked {nbr_itemss} items from this location. Please upload a picture of these items.')
    return PHOTO

def photo(update: Update, context: CallbackContext):
    user = update.message.from_user
    photo_file = update.message.photo[-1].get_file()
    # photo_file.download('user_photo.jpg')

    # Get link to get file_path
    link1 = "https://api.telegram.org/bot{}/getfile?file_id={}".format("5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos", photo_file.file_id)
    r = requests.get(link1)
    file_path = r.json()["result"]["file_path"]

    # Link to download file
    link2 = "https://api.telegram.org/file/bot{}/{}".format("5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos", file_path)
    global img_link
    img_link = link2
    logger.info("Picture: %s", img_link)
    
    client = gspread.authorize(creds)
    sheet = client.open('sample').sheet1
    data = sheet.get_all_records()
    
    # initialize Nominatim API for location address
    geolocator = Nominatim(user_agent="geoapiExercises")

    #inserting the necessary data into a unique variable.
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H:%M:%S")
    rider_name = user.first_name
    geocode = gps_location
    address = geolocator.reverse(gps_location[1:-1]).address
    delivery_number = delivery_numbers
    pick_delivery = pick_deliveries
    nbr_items = nbr_itemss
    img_link  =  img_link

    #inserting the varables into a googlesheet
    row_to_insert = [date, time, rider_name, geocode, address, delivery_number, pick_delivery, nbr_items, img_link]
    sheet.insert_row(row_to_insert, len(data) + 2) # To the amaount of data rows, add 1 for the header, and write 1 below
    
    #sending the data to an email address
    mail(rider_name, location, delivery_number, pick_delivery, nbr_items, img_link)
    update.message.reply_text('Your trip details has been uploaded. Thank you for your cooperation.\n You can start /start_tracking again.')
    return ConversationHandler.END 


def cancel(update: Update, context: CallbackContext):
    user = update.message.from_user
    logger.info("Driver ID %s decided to cancel location record.", user.first_name, update.message.text)

    update.message.reply_text('Location record has been cancelled.\n You can start /start_tracking again.')

    return ConversationHandler.END

def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

#_____________________________________________________________________________________________________

def main():
    '''Start Bot'''
    
    # Launch Updater
    updater = Updater("{}".format("5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos"), use_context=True)
    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # conversions handled by command
    dispatcher.add_handler(CommandHandler("start", start))

    # conversation handler
    loc_handler = ConversationHandler(
        entry_points = [CommandHandler('start_tracking', start_tracking)],
        states = {
            LOC: [MessageHandler(Filters.location, location)],
            PHOTO: [MessageHandler(Filters.photo, photo)],
            DN:[MessageHandler(Filters.text & ~Filters.command, delivery_number)],
            P_D: [MessageHandler(Filters.text & ~Filters.command, pick_delivery)],
            N_I: [MessageHandler(Filters.text & ~Filters.command, nbr_item)],           
        },
        fallbacks = [CommandHandler('cancel', cancel)]
    )
    dispatcher.add_handler(loc_handler)


    # log all errors
    dispatcher.add_error_handler(error)
    # Start the Bot
    updater.start_polling()
    updater.idle()


if __name__ == '__main__':
    main()

2022-07-25 13:31:56,202 - apscheduler.scheduler - INFO - Scheduler started
2022-07-25 13:32:10,535 - __main__ - INFO - Driver: Johnalytics
2022-07-25 13:32:37,401 - __main__ - INFO - Driver ID location Johnalytics: (6.449368, 3.469435)
2022-07-25 13:32:45,395 - __main__ - INFO - Delivery Number: 23
2022-07-25 13:32:53,396 - __main__ - INFO - Job Type: 4
2022-07-25 13:33:01,696 - __main__ - INFO - Number of items: 1
2022-07-25 13:33:10,334 - __main__ - INFO - Picture: https://api.telegram.org/file/bot5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos/photos/file_21.jpg


Mail Sent


2022-07-25 13:33:46,715 - __main__ - INFO - Driver: Johnalytics
2022-07-25 13:34:49,521 - __main__ - INFO - Driver ID location Johnalytics: (6.449383, 3.469436)
2022-07-25 13:35:26,054 - __main__ - INFO - Delivery Number: 2
2022-07-25 13:35:32,579 - __main__ - INFO - Job Type: Delivery
2022-07-25 13:35:42,314 - __main__ - INFO - Number of items: 20
2022-07-25 13:35:52,274 - __main__ - INFO - Picture: https://api.telegram.org/file/bot5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos/photos/file_22.jpg


Mail Sent


2022-07-25 15:17:47,125 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connectionpool.HTTPSConnectionPool object at 0x00000219DDD70C10>, 'Connect timed out. (connect timeout=5.0)')': /bot5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos/getUpdates
2022-07-25 15:17:55,804 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connectionpool.HTTPSConnectionPool object at 0x00000219DDD70C10>, 'Connect timed out. (connect timeout=5.0)')': /bot5510145591:AAEtm0AUhWNEno55TAIpzwVIi3Iu0tHDXos/getUpdates
2022-07-25 15:18:01,862 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None)) after connection 